In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'
import datetime as dt

import time

In [28]:
# Import sign stealing data as astro
path = r'C:\Users\hard\drive\filepath\astros_bangs_20200127.xlsx'pd.set_option('display.max_columns', None)
astro = pd.read_excel(path)
astro.tail(5)

,Line ID,game_id,game_pk,game_date,opponent,final_away_runs,final_home_runs,inning,top_bottom,batter,at_bat_event,pitch_type_code,pitch_category,has_bangs,bangs,call_code,description,on_1b,on_2b,on_3b,pitch_youtube_seconds,youtube_url,pitch_datetime,game_pitch_id,event_number,pitch_playid,atbat_playid,away_team_id,home_team_id
8269,8270,2017_09_24_anamlb_houmlb_1,492424,2017-09-24,ANA,7,5,9,bottom,Jose Altuve,Strikeout,FF,FB,n,NaN,C,Called Strike,f,f,f,14176,https://www.youtube.com/watch?v=oIChAX54tS4&t=...,2017-09-24 22:56:22-05,170925035627,694,3fb6c165-a172-441f-811c-e6f69b87d184,121bea48-7a5d-44e5-b71a-0f16033fc71a,108,117
8270,8271,2017_09_24_anamlb_houmlb_1,492424,2017-09-24,ANA,7,5,9,bottom,Jose Altuve,Strikeout,FF,FB,n,NaN,C,Called Strike,f,f,f,14200,https://www.youtube.com/watch?v=oIChAX54tS4&t=...,2017-09-24 22:56:46-05,170925035651,695,b5be7076-a036-4b98-a988-d644da58f175,121bea48-7a5d-44e5-b71a-0f16033fc71a,108,117
8271,8272,2017_09_24_anamlb_houmlb_1,492424,2017-09-24,ANA,7,5,9,bottom,Jose Altuve,Strikeout,FF,FB,n,NaN,B,Ball,f,f,f,14229,https://www.youtube.com/watch?v=oIChAX54tS4&t=...,2017-09-24 22:57:15-05,170925035720,696,861ed82a-579c-4860-addf-3eb43db80bf3,121bea48-7a5d-44e5-b71a-0f16033fc71a,108,117
8272,8273,2017_09_24_anamlb_houmlb_1,492424,2017-09-24,ANA,7,5,9,bottom,Jose Altuve,Strikeout,FS,FB,n,NaN,B,Ball,f,f,f,14265,https://www.youtube.com/watch?v=oIChAX54tS4&t=...,2017-09-24 22:57:51-05,170925035756,697,fab219e6-64db-4ae8-8e0b-4e040a72dc29,121bea48-7a5d-44e5-b71a-0f16033fc71a,108,117
8273,8274,2017_09_24_anamlb_houmlb_1,492424,2017-09-24,ANA,7,5,9,bottom,Jose Altuve,Strikeout,FF,FB,n,NaN,S,Swinging Strike,f,f,f,14303,https://www.youtube.com/watch?v=oIChAX54tS4&t=...,2017-09-24 22:58:29-05,170925035834,698,121bea48-7a5d-44e5-b71a-0f16033fc71a,121bea48-7a5d-44e5-b71a-0f16033fc71a,108,117


In [29]:
# Dropping columns
astro.drop(['pitch_youtube_seconds','youtube_url','pitch_playid'], axis=1,inplace=True)


Data Exploration

In [30]:
astro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8274 entries, 0 to 8273
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Line ID          8274 non-null   int64         
 1   game_id          8274 non-null   object        
 2   game_pk          8274 non-null   int64         
 3   game_date        8274 non-null   datetime64[ns]
 4   opponent         8274 non-null   object        
 5   final_away_runs  8274 non-null   int64         
 6   final_home_runs  8274 non-null   int64         
 7   inning           8274 non-null   int64         
 8   top_bottom       8274 non-null   object        
 9   batter           8274 non-null   object        
 10  at_bat_event     8274 non-null   object        
 11  pitch_type_code  8274 non-null   object        
 12  pitch_category   8274 non-null   object        
 13  has_bangs        8274 non-null   object        
 14  bangs            1142 non-null   object 

In [31]:
astro.describe()

,Line ID,game_pk,final_away_runs,final_home_runs,inning,event_number,away_team_id,home_team_id
count,8274.000000,8274.000000,8274.000000,8274.000000,8274.000000,8274.000000,8274.000000,8274.0
mean,4137.500000,491202.410442,4.169326,4.943679,4.803481,313.533358,129.169688,117.0
std,2388.642397,707.168856,2.915212,3.565209,2.589308,185.640807,13.361761,0.0
min,1.000000,490111.000000,0.000000,0.000000,1.000000,17.000000,108.000000,117.0
25%,2069.250000,490528.000000,2.000000,3.000000,3.000000,157.000000,116.000000,117.0
50%,4137.500000,491275.000000,4.000000,4.000000,5.000000,297.000000,136.000000,117.0
75%,6205.750000,491752.000000,6.000000,7.000000,7.000000,458.000000,140.000000,117.0
max,8274.000000,492424.000000,13.000000,16.000000,13.000000,908.000000,147.000000,117.0


Jose Altuve

** The focus of this analysis is to look at the 2017 AL MVP Jose Altuve, and determine if stealing signs improved his play. Mainly this will be done by looking at the same set of analyitcal stats from part 4. However we will also look at frequency of use, and outcomes.

In [32]:
# Start by creating a Dataframe were batter is equal to Jose Altuve.
tuve = astro[astro['batter'] == 'Jose Altuve']
#Droping more columns
tuve.drop(['pitch_datetime','game_pitch_id','game_pk','top_bottom','batter','away_team_id','home_team_id'],axis=1,inplace=True)

tuve

,Line ID,game_id,game_pk,game_date,opponent,final_away_runs,final_home_runs,inning,top_bottom,batter,at_bat_event,pitch_type_code,pitch_category,has_bangs,bangs,call_code,description,on_1b,on_2b,on_3b,pitch_datetime,game_pitch_id,event_number,atbat_playid,away_team_id,home_team_id
5,6,2017_04_03_seamlb_houmlb_1,490111,2017-04-03,SEA,0,3,1,bottom,Jose Altuve,Strikeout,SI,FB,n,NaN,B,Ball,t,f,f,2017-04-03 19:27:22-05,170404002722,44,0813793b-63a1-4179-b681-675180afe82d,136,117
6,7,2017_04_03_seamlb_houmlb_1,490111,2017-04-03,SEA,0,3,1,bottom,Jose Altuve,Strikeout,SL,BR,y,1B,C,Called Strike,t,f,f,2017-04-03 19:27:46-05,170404002747,45,0813793b-63a1-4179-b681-675180afe82d,136,117
7,8,2017_04_03_seamlb_houmlb_1,490111,2017-04-03,SEA,0,3,1,bottom,Jose Altuve,Strikeout,FF,FB,n,NaN,F,Foul,t,f,f,2017-04-03 19:28:16-05,170404002817,47,0813793b-63a1-4179-b681-675180afe82d,136,117
8,9,2017_04_03_seamlb_houmlb_1,490111,2017-04-03,SEA,0,3,1,bottom,Jose Altuve,Strikeout,CH,CH,n,NaN,B,Ball,t,f,f,2017-04-03 19:28:45-05,170404002846,48,0813793b-63a1-4179-b681-675180afe82d,136,117
9,10,2017_04_03_seamlb_houmlb_1,490111,2017-04-03,SEA,0,3,1,bottom,Jose Altuve,Strikeout,CH,CH,n,NaN,B,Ball,t,f,f,2017-04-03 19:29:11-05,170404002912,49,0813793b-63a1-4179-b681-675180afe82d,136,117


In [33]:
tuve.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 866 entries, 5 to 8273
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Line ID          866 non-null    int64         
 1   game_id          866 non-null    object        
 2   game_pk          866 non-null    int64         
 3   game_date        866 non-null    datetime64[ns]
 4   opponent         866 non-null    object        
 5   final_away_runs  866 non-null    int64         
 6   final_home_runs  866 non-null    int64         
 7   inning           866 non-null    int64         
 8   top_bottom       866 non-null    object        
 9   batter           866 non-null    object        
 10  at_bat_event     866 non-null    object        
 11  pitch_type_code  866 non-null    object        
 12  pitch_category   866 non-null    object        
 13  has_bangs        866 non-null    object        
 14  bangs            24 non-null     object  

Exploritory analysis of the dataframe tuve.

In [35]:
# Looking at call codes for pitches where a bang was heard.
tuve[tuve['has_bangs'] == 'y']['call_code'].value_counts()

B     6
C     5
X     4
F     2
E     2
*B    2
S     1
W     1
Name: call_code, dtype: int64

Determining the count of bangs used for his At-Bats. 

Based on the yes and no bang count below there were only 24 pitches with bangs in 866 pitches seen.

In [55]:
#Bang comparison of pitches over the 58 games.
tuve['has_bangs'].value_counts()

n    842
y     24
Name: has_bangs, dtype: int64

In [37]:
print(f'In 58 games the percentage of pitches including bangs were: {(24/866)*100} %')

In 58 games the percentage of pitches including bangs were: 2.771362586605081 %


In [38]:
tuve[tuve['has_bangs'] == 'y']['description'].value_counts()

Ball                         6
Called Strike                5
In play, out(s)              4
Foul                         2
In play, run(s)              2
Ball In Dirt                 2
Swinging Strike              1
Swinging Strike (Blocked)    1
Hit By Pitch                 1
Name: description, dtype: int64

In [48]:
#creating a new data frame to show PA outcomes of the 58 game period.
ja = tuve.groupby('atbat_playid').tail(1)

ja

,Line ID,game_id,game_date,opponent,final_away_runs,final_home_runs,inning,at_bat_event,pitch_type_code,pitch_category,has_bangs,bangs,call_code,description,on_1b,on_2b,on_3b,event_number,atbat_playid
10,11,2017_04_03_seamlb_houmlb_1,2017-04-03,SEA,0,3,1,Strikeout,SL,BR,n,NaN,S,Swinging Strike,t,f,f,50,0813793b-63a1-4179-b681-675180afe82d
44,45,2017_04_03_seamlb_houmlb_1,2017-04-03,SEA,0,3,3,Strikeout,SL,BR,n,NaN,W,Swinging Strike (Blocked),t,f,t,157,dff76d36-4e33-4e4f-bdad-4c9dcab56c3d
76,77,2017_04_03_seamlb_houmlb_1,2017-04-03,SEA,0,3,6,Single,FT,FB,n,NaN,D,"In play, no out",t,f,f,314,57ab7cbe-6072-4b56-b6c6-413af4fc78a6
103,104,2017_04_03_seamlb_houmlb_1,2017-04-03,SEA,0,3,8,Flyout,FT,FB,n,NaN,X,"In play, out(s)",f,f,f,452,2db71373-13f3-4bf7-9a4c-bca8ba62ac3e
112,113,2017_04_04_seamlb_houmlb_1,2017-04-04,SEA,1,2,1,Groundout,FC,FB,n,NaN,X,"In play, out(s)",f,f,f,46,025e0899-cfa5-45fd-93fa-89064fc7e3ee
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8144,8145,2017_09_24_anamlb_houmlb_1,2017-09-24,ANA,7,5,1,Hit By Pitch,CU,BR,n,NaN,NaN,Hit By Pitch,f,f,f,52,11542826-52d9-4d6d-b3f2-ca301f010feb
8183,8184,2017_09_24_anamlb_houmlb_1,2017-09-24,ANA,7,5,3,Walk,CH,CH,n,NaN,*B,Ball In Dirt,f,f,f,201,52ed0770-5933-4ed3-a6c3-bb53f4ebb6b0
8214,8215,2017_09_24_anamlb_houmlb_1,2017-09-24,ANA,7,5,5,Single,CU,BR,n,NaN,D,"In play, no out",f,f,f,382,eb66badd-d70e-4448-9022-7a68036eb96f
8238,8239,2017_09_24_anamlb_houmlb_1,2017-09-24,ANA,7,5,7,Single,SL,BR,n,NaN,D,"In play, no out",f,f,f,554,a5696a7f-bc3e-48e9-bb6b-833e2cda928d


In [49]:
# number of games played in
ja['game_date'].nunique()

57

In [51]:
#Looking at PA outcomes where a bang was heard.
ja_y = ja[ja['has_bangs'] == 'y']
# Double and Single are considered positive outcomes.
# out of the 8 banged pitch deciding AB's 2 led to positive outcomes.
# 2 Runs Scored

ja_y


,Line ID,game_id,game_date,opponent,final_away_runs,final_home_runs,inning,at_bat_event,pitch_type_code,pitch_category,has_bangs,bangs,call_code,description,on_1b,on_2b,on_3b,event_number,atbat_playid
302,303,2017_04_05_seamlb_houmlb_1,2017-04-05,SEA,3,5,6,Strikeout,KC,BR,y,1B,S,Swinging Strike,f,f,f,356,c81c5b08-61a0-4a61-a60b-469db5d9354a
993,994,2017_04_17_anamlb_houmlb_1,2017-04-17,ANA,0,3,8,Double,FF,FB,y,1B,E,"In play, run(s)",f,t,f,492,4041e952-457e-4eb9-bcc6-14ecf1568272
1230,1231,2017_04_19_anamlb_houmlb_1,2017-04-19,ANA,1,5,6,Flyout,SL,BR,y,1B,X,"In play, out(s)",f,t,f,338,16a6e554-3b29-44f7-a735-c818c48c393c
3665,3666,2017_06_12_texmlb_houmlb_1,2017-06-12,TEX,6,1,4,Pop Out,CU,BR,y,3B,X,"In play, out(s)",f,f,f,225,e494e950-e813-4204-be95-09117ccde99f
6878,6879,2017_08_22_wasmlb_houmlb_1,2017-08-22,WAS,4,3,1,Flyout,CH,CH,y,1B,X,"In play, out(s)",f,t,f,49,01dc4bd6-bd33-4d01-8766-10f7e0c62475
7425,7426,2017_09_03_nynmlb_houmlb_1,2017-09-03,NYM,6,8,7,Hit By Pitch,SI,FB,y,1B,NaN,Hit By Pitch,f,f,f,545,9de251f9-d927-4e73-a49a-9f29e207a7a8
7681,7682,2017_09_19_chamlb_houmlb_1,2017-09-19,CWS,1,3,6,Lineout,FF,FB,y,1B,X,"In play, out(s)",t,f,f,351,19a98ebe-afef-4656-90e1-613bf06bf476
7833,7834,2017_09_20_chamlb_houmlb_1,2017-09-20,CWS,3,4,7,Single,SL,BR,y,1B,E,"In play, run(s)",f,f,t,442,d4e53df9-e343-4be1-bb11-aa2e327ff143


** From the above Dataframe there are only two positive outcomes where a bang was heard in 241 Plate Appearances.

Calculating the same stat line as in part 4, but just for Jose Altuve over the 58 game spread.

** Calculating baseball statistics; Batting Average(BA), On-Base Percentage (OBP), Slugging percentage (SLG), On-Base Percentage + Slugging (OPS), Weighted On-Base Average (wOBA)

In [41]:
# BA = Hits/AB's
# OBP (H+BB+HBP)/(At Bats + BB + HBP + SF)
# SLG = (1B + 2*B + 3*3B +4*HR)/AB
# OPS = OBP + SLG
# wOBA = (0.690×uBB + 0.722×HBP + 0.888×1B + 1.271×2B + 1.616×3B + 2.101×HR) / (AB + BB – IBB + SF + HBP)

In [42]:
# Using the same strategy from part four

AB = ja[(ja['at_bat_event'] == 'Double') | (ja['at_bat_event'] == 'Home Run') | (ja['at_bat_event'] == 'Single') | (ja['at_bat_event'] == 'Strikeout') | (ja['at_bat_event'] == 'Triple') | (ja['at_bat_event'] == 'Double Play')| (ja['at_bat_event'] == 'Grounded Into DP') |
(ja['at_bat_event'] == 'Flyout') | (ja['at_bat_event'] == 'Groundout') | (ja['at_bat_event'] == 'Lineout') | (ja['at_bat_event'] == 'Field Error') | (ja['at_bat_event'] == 'Runner Out') | (ja['at_bat_event'] == 'Forceout') | (ja['at_bat_event'] == 'Pop Out') | 
(ja['at_bat_event'] == 'Strikeout - DP') | (ja['at_bat_event'] == 'Fielders Choice Out') | (ja['at_bat_event'] == 'Bunt Groundout') | (ja['at_bat_event'] == 'Bunt Pop Out') | (ja['at_bat_event'] == 'Batter Interference') | (ja['at_bat_event'] == 'Fan Interference')].count()

In [43]:
#ja[(ja['at_bat_event'] != 'Walk') | (ja['at_bat_event'] != 'Hit by Pitch') | (ja['at_bat_event'] != 'Sac Fly')]

In [44]:
#Plate Appearances
PA = 241

# At Bats
AB = 216
# Walks
BB = ja[ja['at_bat_event'] == 'Walk'].count()
BB = BB['Line ID'].sum()
#BB

# Strikeout
SO = ja[ja['at_bat_event'] == 'Strikeout'].count()
SO = SO['Line ID'].sum()
#SO

# Hit By Pitch
HBP = ja[ja['at_bat_event'] == 'Hit By Pitch'].count()
HBP = HBP['Line ID'].sum()
#HBP

# Sac Fly
SF = ja[ja['at_bat_event'] == 'Sac Fly'].count()
SF = SF['Line ID'].sum()
#SF

# Single
S = ja[ja['at_bat_event'] == 'Single'].count()
S = S['Line ID'].sum()
#S

# Double
DO = ja[ja['at_bat_event'] == 'Double'].count()
DO = DO['Line ID'].sum()
#DO

#Triple
TR = ja[ja['at_bat_event'] == 'Triple'].count()
TR = TR['Line ID'].sum()
#TR
#Home Runs
HR = ja[ja['at_bat_event'] == 'Home Run'].count()
HR = HR['Line ID'].sum()
#HR

#Hits
H = S+DO+TR+HR


In [45]:
BA = H/AB
OBP = (H+BB+HBP)/(AB + BB + HBP + SF)
SLG = (S + 2*DO + 3*TR + 4*HR)/AB
OPS = OBP + SLG 
wOBA = (0.690*BB + 0.722*HBP + 0.888*S + 1.271*DO + 1.616*TR + 2.101*HR) / (AB + BB + SF + HBP)
print(f'Batting Average for 2017 cheating scandal {BA}')
print(f'On Base Percentage for 2017 cheating scandal {OBP}')
print(f'Slugging Percentage for 2017 cheating scandal {SLG}')
print(f'On Base + Slugging (OPS) for 2017 cheating scandal {OPS}')
print(f'Weighted On-base Average for 2017 cheating scandal {wOBA}')

Batting Average for 2017 cheating scandal 0.2777777777777778
On Base Percentage for 2017 cheating scandal 0.34439834024896265
Slugging Percentage for 2017 cheating scandal 0.4166666666666667
On Base + Slugging (OPS) for 2017 cheating scandal 0.7610650069156293
Weighted On-base Average for 2017 cheating scandal 0.3367053941908713


In [50]:
stat_line = pd.DataFrame({'AB':AB,'BB':BB,'SO':SO,'HBP':HBP,'SF':SF, 'H':H, '1B':S,'2B':DO,'3B':TR,'HR':HR,'BA':BA,'OBP':OBP,'SLG':SLG,'OPS':OPS,'wOBA':wOBA}, index=[0])
stat_line = stat_line.round({'OBP':3,'SLG':3,'OPS':3,'wOBA':3, 'BA':3})
stat_line

,AB,BB,SO,HBP,SF,H,1B,2B,3B,HR,BA,OBP,SLG,OPS,wOBA
0,216,18,39,5,2,60,43,10,1,6,0.278,0.344,0.417,0.761,0.337
